In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#import pickle
path = r'/Users/docopeland/04 Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','prepared data','ords_prods_freq.pkl'))
#view df
ords_prods_merge.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,Busiest 2 Days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regulary busy,Average orders
1,2398795,1,2,3,7,15.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy 2,Average orders
2,473747,1,3,3,12,21.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy 2,Most orders
3,2254736,1,4,4,7,29.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy 2,Average orders
4,431534,1,5,4,15,28.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy 2,Most orders


In [3]:
#finding the mean for order_number grouped by department_id
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})
#this returns all of the department_ids whereas the subsett (seen below) only returns departments 4,7,13,1416,17,19,20

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [4]:
#finding the mean for the subset of data
ords_prods_merge[:1000000].groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [7]:
#create a new column that shows max number of orders per user
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)
ords_prods_merge.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,Busiest 2 Days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regulary busy,Average orders,10
1,2398795,1,2,3,7,15.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy 2,Average orders,10
2,473747,1,3,3,12,21.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Least busy 2,Most orders,10
3,2254736,1,4,4,7,29.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy 2,Average orders,10
4,431534,1,5,4,15,28.0,False,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Least busy 2,Most orders,10


In [8]:
#setting up my loyalty flags
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [28]:
#group the prices by loyalty_flag
#new custoemrs send the most, loyal customers spend the least on average per product
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['min','mean','median','max']})

prices                           
                    min       mean median      max
loyalty_flag                                      
Loyal customer      1.0  10.386336    7.4  99999.0
New customer        1.0  13.294670    7.4  99999.0
Regular customer    1.0  12.495717    7.4  99999.0

In [14]:
#find the average spend per product per person
ords_prods_merge['user_avg'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [15]:
#create a high spender flag based on 'user_avg'
ords_prods_merge.loc[ords_prods_merge['user_avg'] < 10, 'spender_flag'] = 'Low spender'

In [16]:
ords_prods_merge.loc[ords_prods_merge['user_avg'] >= 10, 'spender_flag'] = 'High spender'

In [17]:
#check to see how many high and low spenders they are
ords_prods_merge['spender_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_flag, dtype: int64

In [19]:
#find median of days_since_prior_order
ords_prods_merge['order_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [20]:
#create a customer frequency flag based on 'order_freq'
ords_prods_merge.loc[ords_prods_merge['order_freq'] > 20, 'freq_flag'] = 'Non-frequent customer'

In [21]:
ords_prods_merge.loc[(ords_prods_merge['order_freq'] > 10) & (ords_prods_merge['order_freq'] <= 20), 'freq_flag'] = 'Regular customer'

In [22]:
ords_prods_merge.loc[ords_prods_merge['order_freq'] <= 10, 'freq_flag'] = 'Frequent customer'

In [23]:
#check to see the frequency flags
ords_prods_merge['freq_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: freq_flag, dtype: int64

In [26]:
#check to see the order frequencies
#not sure why there are NaNs
ords_prods_merge['order_freq'].value_counts(dropna = False)

7.0     5682399
6.0     2983604
8.0     2713964
5.0     2486061
4.0     2314289
         ...   
26.5       5514
29.5       5095
1.5        1684
0.5         235
NaN           5
Name: order_freq, Length: 62, dtype: int64

In [27]:
ords_prods_merge.to_pickle(os.path.join(path,'02 Data','prepared data','ords_prods_freq_flags.pkl'))